# Correlation analysis between chlorophyll a (indicator for marine plants) and seawater temperature 

This report analyzes a possible correlation between chlorophyll a in seawater which indicates changes in marine plants like phytoplankton and seawater temperature. It uses open data from Mobilithek (https://data.bsh.de/OpenData/DOD/MO_H_CHLA/ and https://data.bsh.de/OpenData/DOD/MO_H_TEMP/) for the analysis.
The data is licensed under following license:
License: GeoSeaPortal des BSH, licensed under the following license: Data licence Germany – attribution – Version 2.0" or "dl-de/by-2-0" referring to the licence text available at www.govdata.de/dl-de/by-2-0, data available at https://data.bsh.de/OpenData/DOD/MO_H_CHLA/ 

## Install dependencies
Initially, install all required dependencies. 

In [ ]:
%pip install pandas
%pip install matplotlib
%pip install numpy

## Load data
Create a pandas dataframe using the local sqlite file.

In [22]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import math

con = sqlite3.connect('./data/data.sqlite')
df1 = pd.read_sql_query("SELECT * FROM CHLA_18", con)
df2 = pd.read_sql_query("SELECT * FROM CHLA_22", con)
df3 = pd.read_sql_query("SELECT * FROM TEMP_18", con)
df4 = pd.read_sql_query("SELECT * FROM TEMP_22", con)

## How have the chlorophyl a and temperature levels in the North and Baltic Seas of Germany changed?
Four graphs are generated: chlorophyl a + temperature 2018, chlorophyll a + temperature 2022, chlorophyll a + temperature changes from 2018 to 2022, chlorophyll a + temperature by station. For the graphs, the measurement depth is also taken into account, so there might be even more graphs.

In [31]:
# read data for chlorophyl a from 2018 
pd.to_datetime(df1.Date)
# combine values from different measuring methods
avg=[]
for u in range(len(df1)):
    i=0
    if  math.isnan(df1["CHLA_SENE_DEV_NDT [µg/l]"][u]):
        i+=1
    if not math.isnan(df1["CHLA_FLM_FGN_AL [µg/l]"][u]):
        i+=1
    if i != 0:
        avg.append(df1["CHLA_SENE_DEV_NDT [µg/l]"][u] + df1[ "CHLA_FLM_FGN_AL [µg/l]"][u]/i)
    else:
        avg.append(math.nan)
# df1 = df1.assign(avg=avg)  
# combine values from same mont
# df1.groupby(pd.PeriodIndex(df1['Date'], freq="M"))["avg"].mean()
# use the plot function
# plt.plot(values)